In [1]:
%matplotlib widget
import os
import numpy as np
# from scipy import linalg, spatial, fft, signal
# from sklearn.experimental import enable_iterative_imputer
# from sklearn import metrics, decomposition, impute, preprocessing
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib import colors, gridspec
import seaborn as sns
# from matplotlib.patches import RegularPolygon, Rectangle
# from PIL import Image, ExifTags, ImageFile
import visualize as vis
# import CR_search as cr
import preprocessing as pp
import calculate as cc
from read_WDF_class import WDF

# ImageFile.LOAD_TRUNCATED_IMAGES = True

folder = os.path.expanduser("~/Documents/RamanData/")
subfolder = "frejus21"
files = os.listdir(os.path.join(folder,subfolder))

for i, tf in enumerate(files):
    if tf[-3:] == 'wdf':
        print(i, ') ', tf)  

0 )  exampleM1.wdf
1 )  exampleA1.wdf
2 )  exampleM2.wdf
3 )  exampleM3.wdf
4 )  exampleA2.wdf
6 )  exampleM4.wdf


In [2]:
file = files[1]
filename = os.path.join(folder,subfolder,file)

fff = WDF(filename, verbose=True)

Reading the file: "exampleA1.wdf"


================================= Block : WDF1 =================================
size: 512, offset: 0
WdfFlag--------------------------------- : 	WdfXYXY
PointsPerSpectrum----------------------- : 	1015
Capacity-------------------------------- : 	2006
Count----------------------------------- : 	2006
AccumulationCount----------------------- : 	1
YlistLength----------------------------- : 	1
XlistLength----------------------------- : 	1015
DataOriginCount------------------------- : 	9
ApplicationName------------------------- : 	WiRE
ApplicationVersion---------------------- : 	5.3.0 build 16195
ScanType-------------------------------- : 	StreamLineHR
MeasurementType------------------------- : 	Map
StartTime------------------------------- : 	Mon Jun 21 08:06:02 2021
EndTime--------------------------------- : 	Mon Jun 21 12:35:25 2021
SpectralUnits--------------------------- : 	Counts
LaserWaveLength------------------------- : 	532.06
Title---------------

### Here are some of the treatments you might apply to your input spectra:
Do this first:
```python
fff = pp.order(fff)
```
Normalizing:
```python
fff = pp.normalize(fff)
```
Selecting the zone of interest:
```python
fff = pp.select_zone(fff, left=125, right=None)
```
Removing the Cosmic Rays:
```python
fff = pp.remove_CRs(fff)
```
Removing the baseline:
(don't do this on the cloud!)
```python
baseline = cc.baseline_als(fff.spectra, p=0.001)
```
Clean the spectra with the PCA:
```python
fff = pp.pca_clean(fff, n_components=21)
```

In [3]:
sns.set_style("dark")
show1 = vis.ShowSelected(fff)
show1.fig.set_size_inches(8, 6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
fff.show_grid(alpha=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
vis.AllMaps(fff);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
fff = pp.remove_CRs(fff)

In [5]:
# =============================================================================
#                                     PCA...
# =============================================================================
fff = pp.pca_clean(fff, n_components=25)

In [6]:
fff = pp.normalize(fff)

In [7]:
mix, components, reconstucted_spectra = cc.deconvolute_nmf(fff, 4)

/home/dejan/anaconda3/envs/Raman/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 7 reached. Increase it to improve convergence.
  warnings.warn(


In [8]:
_n_components = len(components)
_comp_area = np.empty(_n_components)
for _z in range(_n_components):
    # area beneath each component:
    _comp_area[_z] = np.trapz(components[_z])#, x=fff.x_values)
    components[_z] /= _comp_area[_z]  # normalizing the components by area
    # renormalizing the mixture coefficients:
    mix[:, _z] *= _comp_area[np.newaxis, _z]
mix /= np.sum(mix, axis=-1)[:,np.newaxis]
spectra_reconstructed = np.dot(mix, components)
_mix_reshaped = mix.reshape(fff.n_y, fff.n_x, _n_components)

col_norm = colors.Normalize(vmin=0, vmax=_n_components)
color_set = ScalarMappable(norm=col_norm, cmap="brg")

# infer the number of subplots and their disposition from n_components
fi, _ax = plt.subplots(int(np.floor(np.sqrt(_n_components))),
                       int(np.ceil(_n_components /
                                   np.floor(np.sqrt(_n_components))
                                   )))
if _n_components > 1:
    _ax = _ax.ravel()
else:
    _ax = [_ax]
for _i in range(_n_components):
    _ax[_i].plot(fff.x_values, components[_i].T, color=color_set.to_rgba(_i))
    _ax[_i].set_title(f'Component {_i}')
    _ax[_i].set_yticks([])
try:
    fi.text(0.5, 0.04,
            f"{params['XlistDataType']} recordings"
            f"in {params['XlistDataUnits']} units",
            ha='center')
except:
    pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# %%
# =============================================================================
#                       Plotting the main plot...(heatmap)
# =============================================================================
_start_pos = 0
_n_fig_rows = int(np.floor(np.sqrt(_n_components)))
_n_fig_cols = int(np.ceil(_n_components / np.floor(np.sqrt(_n_components))))
gs1 = gridspec.GridSpec(_n_fig_rows, _n_fig_cols, left=0.05, right=0.48, wspace=0.05)
fig, _ax = plt.subplots(_n_fig_rows, _n_fig_cols,
                        sharex=True, sharey=True)
if _n_components > 1:
    _ax = _ax.ravel()
else:
    _ax = [_ax]


def onclick(event):
    '''Double-clicking on a pixel will show the (cleaned) spectrum
    corresponding to that pixel, as well as its deconvolution on the components
    and again the reconstruction for visual comparison'''
    if event.inaxes:
        x_pos = round(event.xdata)
        y_pos = round(event.ydata)
        broj = round(y_pos*fff.n_x + x_pos)
        spec_num = np.round(y_pos*fff.n_x - _start_pos + x_pos)

        if event.button!=1:
            ff, aa = plt.subplots()
            aa.scatter(fff.x_values, fff.spectra[spec_num], alpha=0.3,
                       label=f'(cleaned) spectrum n°{broj}')
            aa.plot(fff.x_values, spectra_reconstructed[broj], '--k',
                    label='reconstructed spectrum')
            for k in range(_n_components):
                aa.plot(fff.x_values, components[k]*mix[broj][k],
                        color=color_set.to_rgba(k),
                        label=f'Component {k} contribution'
                              f'({mix[broj][k]*100:.1f}%)')

# This next part is to reorganize the order of labels,
# so to put the scatter plot first
            handles, labels = aa.get_legend_handles_labels()
            order = list(np.arange(_n_components+2))
            new_order = [order[-1]]+order[:-1]
            aa.legend([handles[idx] for idx in new_order],
                      [labels[idx] for idx in new_order])
            aa.set_title(f'deconvolution of the spectrum from: '
                         f'line {y_pos} & column {x_pos}')
            ff.show()
    else:
        print("you clicked outside the canvas, you bastard :)")


_xcolumn_name = 'X (Micron)'
_ycolumn_name = 'Y (Micron)'

#################################################################################
############## This formatting should be adapted case by case ###################
try:
    _y_ticks = [str(int(x/1000))+'mm' for x in
                np.asarray(origins[_ycolumn_name].iloc[:fff.n_x*fff.n_y:fff.n_x])]
    _x_ticks = [str(int(x/1000))+'mm' for x in
                np.asarray(origins[_xcolumn_name].iloc[:fff.n_x])]
except:
    pass
#################################################################################


for _i in range(_n_components):
    sns.heatmap(_mix_reshaped[:, :, _i], ax=_ax[_i], cmap="jet", annot=False)
#    _ax[_i].set_aspect(_s_y/_s_x)
    _ax[_i].set_title(f'Component {_i}', color=color_set.to_rgba(_i),
                      fontweight='extra bold')
fig.suptitle('Heatmaps showing the abundance of individual components'
             ' throughout the scanned area.')
fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

9

In [42]:
plt.close('all')